In [161]:
#!g1.1
# !git clone https://github.com/nerel-ds/NEREL

In [162]:
#!g1.1
# !unzip NEREL-v1.0.zip -d NEREL

In [163]:
#!g1.1
## Read the data# Reading files
from collections import namedtuple
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import nltk

Ann = namedtuple('annotation', ['tag', 'start1', 'end1', 'start2', 'end2', 'text'])
Rel = namedtuple('relationship', ['tag', 'arg1', 'arg2'])

def read_files(folder):
    filenames = sorted(set(e[:e.rfind('.')] for e in os.listdir(folder)))[1:]
    
    texts, entities, relationships  = [], [], []
    for file in tqdm(filenames):
       # if (not file): continue
        path1 = os.path.join(folder, file+'.txt')
        path2 = os.path.join(folder, file+'.ann')
        if (not os.path.exists(path2)):
            print(f'{path2} not found')
            continue
            #with open(path2, 'w'):
            #    pass
            
        with open(path1, 'r', encoding="utf8") as text, open(path2, 'r', encoding="utf8") as ann:
            texts.append(text.read())

            file_entities = {}
            file_relationship = []
            regex_ent = r'T(?P<id>\d+)\s(?P<tag>\w+)\s(?P<start1>\d+) (?P<end1>\d+)(;(?P<start2>\d+) (?P<end2>\d+))?\s(?P<text>.*)'
            regex_rel = r'R(?P<id>\d+)\s(?P<tag>\w+)\sArg1:T(?P<arg1>\d+) Arg2:T(?P<arg2>\d+)'
            
            
            for row in sorted(ann.readlines(), reverse=True):
                #print(row)
                match_ent = re.match(regex_ent, row)
                match_rel = re.match(regex_rel, row)
                if (match_ent):
                    res = match_ent.groupdict()
                    res['start1'] = int(res['start1'])
                    res['end1'] = int(res['end1'])
                    if (res['start2'] is not None):
                        res['start2'] = int(res['start2'])
                        res['end2'] = int(res['end2'])
                    id = res.pop('id')
                    file_entities[id] = Ann(**res)
                elif (match_rel):
                    try:
                        res = match_rel.groupdict()
                        res['arg1'] = file_entities[res['arg1']]
                        res['arg2'] = file_entities[res['arg2']]
                        id = res.pop('id')
                        file_relationship.append(Rel(**res))
                    except KeyError as e:
                        print(f'not found T{e} row={row}')
                else:
                    print(f'incorrect format in: row={row} file={file}')
            entities.append(file_entities)
            relationships.append(file_relationship)
    entities = [sorted(e.values(), key = lambda x: (x.start1, x.end1)) for e in entities]
    return texts, entities, relationships, filenames

In [164]:
#!g1.1
folder = 'NEREL/NEREL-v1.0/train'
texts, entities, relationships, filenames = read_files(folder)

incorrect format in: row=По словам очевидцев пешехо
 file=21013_text


100%|██████████| 745/745 [00:01<00:00, 452.36it/s]


In [165]:
#!g1.1
def in_range(range1, range2):
    # range1 in range2
    if range1[0] is None: return True
    if range2[0] is None: return False
    return int(range2[0]) <= int(range1[0]) and int(range2[1]) >= int(range1[1])

def is_nested_anns(ent1: Ann, ent2: Ann):
    # ent 1 in ent 2
    res = True
    ent1_range1 = (ent1.start1, ent1.end1)
    ent1_range2 = (ent1.start2, ent1.end2)
    ent2_range1 = (ent2.start1, ent2.end1)
    ent2_range2 = (ent2.start2, ent2.end2)
    #print(f'{ent1_range1=} {ent1_range2=} {ent1_range2=} {ent2_range2=}')
    res = res and (in_range(ent1_range1, ent2_range1) or in_range(ent1_range1, ent2_range2))
    res = res and (in_range(ent1_range2, ent2_range1) or in_range(ent1_range2, ent2_range2))
    return res


def is_nested_anns2(ent1: Ann, ent2: Ann):
    return is_nested_anns(ent1,ent2) or is_nested_anns(ent2, ent1)

def is_nested(rel: Rel):
    return is_nested_anns(rel.arg1, rel.arg2) or is_nested_anns(rel.arg2, rel.arg1)

![](https://i.imgur.com/tgDfc8i.png)             | ![](https://i.imgur.com/oWa5vWo.png)
:-------------------------:|:-------------------------:


In [166]:
#!g1.1
%pip install transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [167]:
#!g1.1
# pre_dataset = []

# for text_id in range(len(texts)):
#     relationships_nested = {(e.arg1, e.arg2):e for e in relationships[text_id] if is_nested(e)}
#     nes = []
#     for i in range(len(entities[text_id])):
#         # O(n^2) eeeeeeeeee
#         for j in range(i+1, len(entities[text_id])):
#             if (is_nested_anns2(entities[text_id][i], entities[text_id][j])):
#                 nes.append((entities[text_id][i], entities[text_id][j]))
#     for e in nes:
#         if (e in relationships_nested):
#             pre_dataset.append((*e, relationships_nested[e].tag))
#         elif ((e[1], e[0]) in relationships_nested):
#             pre_dataset.append((e[1], e[0], relationships_nested[(e[1], e[0])].tag))
#         else:
#             pre_dataset.append((*e, 'None'))
# #             pre_dataset.append((e[1], e[0], 'None'))

# Dataset

In [168]:
#!g1.1
import os
import torch
import time
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from tqdm.notebook import tqdm
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from matplotlib import pyplot as plt
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [169]:
#!g1.1
from transformers import BertForSequenceClassification, BertTokenizer

In [170]:
#!g1.1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [171]:
#!g1.1
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

In [172]:
#!g1.1
MAX_LENGTH = 100

In [173]:
#!g1.1
class Binary_Tag:
    def __getitem__(self, x):
        if (x == 'None'): return 0
        else: return 1

# tags = list(set(e.tag for e in sum(relationships,[]))) + ['None'] # ids --> string tag
reversed_tags = Binary_Tag() # string tag --> ids

entity_tags = list(set(e.tag for e in sum(entities,[]))) 

def tok(text1):
    # longest entity is 36 tokens long
    res = tokenizer.encode_plus(text1,
                      max_length = MAX_LENGTH,
                      pad_to_max_length = True,
                      return_tensors = 'pt',
                      )
    return res['input_ids'][0], res['attention_mask'][0], res['token_type_ids'][0]
    
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, folder):
        texts, entities, relationships, filenames = read_files(folder)
        
        pre_dataset = []

        for text_id in range(len(texts)):
            relationships_nested = {(e.arg1, e.arg2):e for e in relationships[text_id] if is_nested(e)}
            nes = []
            for i in range(len(entities[text_id])):
                # O(n^2) eeeeeeeeee
                for j in range(i+1, len(entities[text_id])):
                    if (is_nested_anns2(entities[text_id][i], entities[text_id][j])):
                        nes.append((entities[text_id][i], entities[text_id][j]))
            for e in nes:
                if (e in relationships_nested):
                    pre_dataset.append((*e, relationships_nested[e].tag))
                elif ((e[1], e[0]) in relationships_nested):
                    pre_dataset.append((e[1], e[0], relationships_nested[(e[1], e[0])].tag))
                else:
                    pre_dataset.append((*e, 'None'))

#         random.seed(2021)
        data = []
        for e1, e2, tag in tqdm(pre_dataset):
            tag1, tag2 = entity_tags.index(e1.tag), entity_tags.index(e2.tag)
            e1, e2 = e1.text, e2.text
            if (len(e1) > len(e2)):
                text = e1.replace(e2, '[ ' + e2 + ' ]')
            else:
                text = e2.replace(e1, '[ ' + e1 + ' ]')
            data.append((float(reversed_tags[tag]), tag1, tag2, *tok(text)))
        
        self.data = data
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)
    

In [174]:
#!g1.1
train_data = MyDataset('NEREL/NEREL-v1.0/train')
train_dl = DataLoader(train_data, shuffle=True, batch_size=32)

dev_data = MyDataset('NEREL/NEREL-v1.0/dev')
dev_dl = DataLoader(dev_data, batch_size=32)

incorrect format in: row=По словам очевидцев пешехо
 file=21013_text






In [175]:
#!g1.1
next(iter(dev_dl))

[tensor([0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0.],
        dtype=torch.float64),
 tensor([ 3,  3,  3,  3,  6,  6,  6,  6,  6,  6,  3,  3,  5, 17, 27, 13,  8,  8,
          6,  6,  6,  9, 22, 19,  3,  3,  3,  3,  1,  1,  1,  3]),
 tensor([ 3, 28,  3, 28,  6,  6,  6, 28,  6, 28, 28,  3,  6, 27, 17,  8,  3,  8,
          8,  6, 28, 22, 22,  9,  6,  6,  3, 28, 18, 18, 18,  3]),
 tensor([[   101,    222,  17710,  ...,      0,      0,      0],
         [   101,  17710,    876,  ...,      0,      0,      0],
         [   101,    222,  13796,  ...,      0,      0,      0],
         ...,
         [   101, 113644,  12333,  ...,      0,      0,      0],
         [   101, 113644,  12333,  ...,      0,      0,      0],
         [   101,    222,  11615,  ...,      0,      0,      0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
  

In [176]:
#!g1.1
next(iter(dev_dl))[1].size()

torch.Size([32])

# Training

In [177]:
#!g1.1
# from transformers import BertTokenizer, BertModel
# import torch

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')

# inputs = tokenizer.encode_plus("Hello, my dog is cute", return_tensors="pt")
# outputs = model(**inputs)

# pooled_output = outputs[1]

In [194]:
#!g1.1
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained(
            "DeepPavlov/rubert-base-cased-sentence", # Use the 12-layer BERT model, with an uncased vocab.
            output_attentions = False, # Whether the model returns attentions weights.
            output_hidden_states = False, # Whether the model returns all hidden-states.
            max_length = MAX_LENGTH
            )
        self.dropout = nn.Dropout(0.1)
        self.emb = nn.Embedding(29, 10)
        self.fc = nn.Sequential(
            nn.Linear(768 + 10*2, 100),
            nn.LeakyReLU(True),
            nn.Linear(100, 1)
            )
        
    def forward(self, ent1, ent2, input_ids, token_type_ids, attention_mask):
        ent1 = self.emb(ent1)
        ent2 = self.emb(ent2)
        out = self.bert(input_ids = input_ids, token_type_ids= token_type_ids, attention_mask = attention_mask)[1]
        out = self.dropout(out)
        out = torch.cat((out, ent1, ent2), dim=1)
        out = self.fc(out)
        return out

In [195]:
#!g1.1
model = MyModel()
model.to(device)

MyModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

In [181]:
#!g1.1
# model(torch.tensor([1]), torch.tensor([1]), **tokenizer.encode_plus("привет", return_tensors="pt"))

In [182]:
#!g1.1
tokenizer.encode_plus("привет", return_tensors="pt")

{'input_ids': tensor([[  101, 26856,   102]]),
 'token_type_ids': tensor([[0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1]])}

In [192]:
#!g1.1
def calculate_accuracy(y_pred, y):
    return sum(torch.round(torch.sigmoid(y_pred)) == y)*1.0/len(y)

def train():
    model.train()
    running_loss = 0
    epoch_accuracy = 0

    pbar = tqdm(enumerate(train_dl), total = len(train_dl))
    for i, (labels, tag1, tag2, input_ids, attention_mask, token_type_ids) in pbar:
        labels = labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        tag1 = tag1.to(device)
        tag2 = tag2.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(
                    tag1,
                    tag2,
                    input_ids = input_ids, 
                    token_type_ids=token_type_ids,
                    attention_mask=attention_mask)
        
#         print(outputs.shape, labels.shape)
        outputs = outputs.view(-1)
        loss = criterion(outputs, labels)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
#         print(outputs, labels)
        epoch_accuracy += calculate_accuracy(outputs, labels).item()
        running_loss += loss.item()
        pbar.set_description(f'training: running_loss = {running_loss/(i+1.0):.4f} accuracy = {epoch_accuracy/(i+1.0):.4f},')
    
    scheduler.step(running_loss/(i+1))
    print(f'train loss= {running_loss/(i+1):.4f} \n train accuracy = {epoch_accuracy/(i+1):.4f},')

    
from sklearn.metrics import f1_score
def test():
    with torch.no_grad():
        model.eval()
        running_loss = 0
        epoch_accuracy = 0

        dev_dl = DataLoader(dev_data, batch_size=32)

        pbar = tqdm(enumerate(dev_dl), total = len(dev_dl))

        y_true = []
        y_pred = []

        for i, (labels, tag1, tag2, input_ids, attention_mask, token_type_ids) in pbar:
            labels = labels.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            tag1 = tag1.to(device)
            tag2 = tag2.to(device)
            
            outputs = model(
                    tag1,
                    tag2,
                    input_ids = input_ids, 
                    token_type_ids=token_type_ids,
                    attention_mask=attention_mask)
            
            
            outputs = outputs.view(-1)
            loss = criterion(outputs, labels)
#             calculate_accuracy(outputs[1], labels)
            
            y_true += [e.item() for e in labels]
            y_pred += [e.item() for e in torch.round(torch.sigmoid(outputs))]

            running_loss += loss.item()
            epoch_accuracy += calculate_accuracy(outputs, labels).item()
            # pbar.set_description(f'testing: running_loss = {running_loss/(i+1):.4f} accuracy = {epoch_accuracy/(i+1):.4f},')
    print(f'test loss= {running_loss/(i+1):.4f} \n test accuracy = {epoch_accuracy/(i+1):.4f}, F1 = {f1_score(y_true, y_pred)}')

In [196]:
#!g1.1
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate 
                  eps = 5e-8#1e-8 # args.adam_epsilon 
                )
criterion = nn.BCEWithLogitsLoss()
scheduler = ReduceLROnPlateau(optimizer, patience=5, cooldown = 1, factor = 0.5)

In [197]:
#!g1.1
seed_val = 2021

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
for epoch in range(1, 21):
    print('lr=', optimizer.param_groups[0]['lr'])
    print(f'epoch = {epoch}')
    train()
    test()

lr= 5e-05
epoch = 1

train loss= 0.3528 
 train accuracy = 0.8516,

test loss= 0.3484 
 test accuracy = 0.8563, F1 = 0.8234183191690273
lr= 5e-05
epoch = 2

train loss= 0.2573 
 train accuracy = 0.9107,

test loss= 0.3996 
 test accuracy = 0.8563, F1 = 0.8074150360453141
lr= 5e-05
epoch = 3

train loss= 0.2328 
 train accuracy = 0.9234,

test loss= 0.3717 
 test accuracy = 0.8612, F1 = 0.8132780082987551
lr= 5e-05
epoch = 4

train loss= 0.1914 
 train accuracy = 0.9399,

test loss= 0.4320 
 test accuracy = 0.8345, F1 = 0.760845383759733
lr= 5e-05
epoch = 5

train loss= 0.1801 
 train accuracy = 0.9431,

test loss= 0.3491 
 test accuracy = 0.8753, F1 = 0.8430232558139535
lr= 5e-05
epoch = 6

train loss= 0.1657 
 train accuracy = 0.9487,

test loss= 0.4612 
 test accuracy = 0.8643, F1 = 0.8297872340425532
lr= 5e-05
epoch = 7

train loss= 0.1486 
 train accuracy = 0.9555,

test loss= 0.4131 
 test accuracy = 0.8723, F1 = 0.8316430020283976
lr= 5e-05
epoch = 8

train loss= 0.1398 
 train a

In [198]:
#!g1.1
torch.save(model.state_dict(), 'models/Bert_binary_nested_only_ent_tags')

In [ ]:
# #!g1.1
# # from sklearn.metrics import f1_score

# with torch.no_grad():
#     model.eval()
#     running_loss = 0
#     epoch_accuracy = 0

#     dev_dl = DataLoader(dev_data, batch_size=1)

#     pbar = tqdm(enumerate(dev_dl), total = len(dev_dl))

#     y_true = []
#     y_pred = []

#     for i, (labels, input_ids, attention_mask, token_type_ids) in pbar:
#         labels = labels.to(device)
#         input_ids = input_ids.to(device)
#         attention_mask = attention_mask.to(device)
#         token_type_ids = token_type_ids.to(device)

#         outputs = model(input_ids, 
#                     token_type_ids=token_type_ids,
#                     attention_mask=attention_mask, 
#                     labels=labels)


#         loss = outputs[0]
# #         print(input_ids[0].detach().cpu().numpy())
#         if (torch.argmax(outputs[1], axis = 1) != labels):
#             print('expected:', labels.detach().cpu().item())
#             print('found:', torch.argmax(outputs[1], axis = 1).detach().cpu().item(), '(', outputs[1].detach().cpu().numpy() ,')')
#             print(tokenizer.decode(input_ids[0].detach().cpu().numpy()).replace(r'[PAD]', ''))

In [200]:
#!g1.1

with torch.no_grad():
    model.eval()
    running_loss = 0
    epoch_accuracy = 0

    dev_dl = DataLoader(dev_data, batch_size=1)

    pbar = tqdm(enumerate(dev_dl), total = len(dev_dl))

    y_true = []
    y_pred = []
    tags = []

    for i, (labels, tag1, tag2, input_ids, attention_mask, token_type_ids) in pbar:
        labels = labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        tag1 = tag1.to(device)
        tag2 = tag2.to(device)

        outputs = model(
                tag1,
                tag2,
                input_ids = input_ids, 
                token_type_ids=token_type_ids,
                attention_mask=attention_mask)


        outputs = outputs.view(-1)
        loss = criterion(outputs, labels)
#         calculate_accuracy(outputs[1], labels)

        y_true += [e.item() for e in labels]
        y_pred += [e.item() for e in torch.round(torch.sigmoid(outputs))]
        tags += [(tag1, tag2)]
        
        running_loss += loss.item()
        epoch_accuracy += calculate_accuracy(outputs, labels).item()
        # pbar.set_description(f'testing: running_loss = {running_loss/(i+1):.4f} accuracy = {epoch_accuracy/(i+1):.4f},')
print(f'test loss= {running_loss/(i+1):.4f} \n test accuracy = {epoch_accuracy/(i+1):.4f}, F1 = {f1_score(y_true, y_pred)}')


test loss= 0.5078 
 test accuracy = 0.8409, F1 = 0.7742639040348964


/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:859: UserWarning: The following variables cannot be serialized: pbar
  warnings.warn(message)


In [209]:
#!g1.1
def fun(e):
    return entity_tags[e[0].item()], entity_tags[e[1].item()]
l = []
for true, pred, t in zip(y_true, y_pred, tags):
    if (true != pred):
        l.append(fun(t))

In [215]:
#!g1.1
from collections import Counter
Counter(l).most_common()

[(('PROFESSION', 'ORGANIZATION'), 33),
 (('ORGANIZATION', 'ORGANIZATION'), 18),
 (('EVENT', 'LOCATION'), 10),
 (('ORGANIZATION', 'STATE_OR_PROVINCE'), 8),
 (('ORGANIZATION', 'COUNTRY'), 8),
 (('PROFESSION', 'COUNTRY'), 7),
 (('EVENT', 'EVENT'), 6),
 (('PROFESSION', 'STATE_OR_PROVINCE'), 6),
 (('DISEASE', 'DISEASE'), 5),
 (('EVENT', 'PERSON'), 5),
 (('PERSON', 'ORGANIZATION'), 4),
 (('LAW', 'LAW'), 4),
 (('PROFESSION', 'PROFESSION'), 4),
 (('PERSON', 'EVENT'), 4),
 (('ORGANIZATION', 'CITY'), 4),
 (('PROFESSION', 'LOCATION'), 3),
 (('EVENT', 'DISEASE'), 3),
 (('LAW', 'STATE_OR_PROVINCE'), 3),
 (('ORGANIZATION', 'RELIGION'), 3),
 (('WORK_OF_ART', 'PERSON'), 3),
 (('EVENT', 'CITY'), 3),
 (('EVENT', 'STATE_OR_PROVINCE'), 3),
 (('LOCATION', 'ORGANIZATION'), 2),
 (('PERSON', 'FAMILY'), 2),
 (('LAW', 'COUNTRY'), 2),
 (('PENALTY', 'MONEY'), 2),
 (('PROFESSION', 'EVENT'), 2),
 (('PROFESSION', 'CITY'), 2),
 (('ORGANIZATION', 'PROFESSION'), 2),
 (('ORGANIZATION', 'DISTRICT'), 2),
 (('PERSON', 'FAC

In [ ]:
#!g1.1
